# Text Augmentation


1. Easy Data Augmentation
    - Random Deletion (직접 구현해보기)
    - Random Insertion
    - Random Swap (직접 구현해보기)
    - Synonym Replacement
2. Back Translation (직접 구현해보기)

In [1]:
import pandas as pd
import random
import pickle
import re

In [2]:
!gdown https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

Downloading...
From: https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
To: /content/train_original.json
100% 800M/800M [00:10<00:00, 75.6MB/s]


In [3]:
df_news = pd.read_json("./train_original.json")
df_news

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"
...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]"
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]"
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]"
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]"


In [4]:
train = df_news[['abstractive']].loc[:100]
train.head()

,abstractive
0,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...
1,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...
2,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...
3,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...
4,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...


In [7]:
sample = train.values[0, 0]
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

### Random Deletion

< 사용할 함수 >   
- random.uniform(0, 1)
- random.randint(0, n)

In [29]:
# 정한 확률만큼 deletion = 문장을 공백으로 분할하여 단어 목록을 만듭니다.

def random_deletion(sent, p):
	# 문장을 공백으로 분할하여 단어 목록을 만듭니다.
    words = sent.split()

    # 단어 목록에서 각 단어를 제거할지 여부를 결정합니다.
    filtered_words = [word for word in words if random.uniform(0, 1) > p]

    # 제거된 단어를 다시 하나의 문장으로 결합합니다.
    result_sent = ' '.join(filtered_words)

    return result_sent

### Random Swap

In [30]:
# 정한 갯수만큼 swap =  정해진 갯수 n만큼 단어를 무작위로 교환하는 random_swap 함수
def random_swap(sent, n):
    # 문장을 공백으로 분할하여 단어 목록을 만듭니다.
    words = sent.split()

    # 교환할 단어 쌍을 무작위로 선택하여 교환합니다.
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]

    # 교환된 단어를 다시 하나의 문장으로 결합합니다.
    result_sent = ' '.join(words)

    return result_sent

## Back Translation

In [5]:
!pip install --upgrade translators

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14578 sha256=25a35773063682e14650e0310ec4ab494a565e9eab2d6f821b834d7f7a51e4c3
  Stored in directory: /root/.cache/pip/wheels/9d/91/30/28e6da53d4f44dc445349b2ffad581968447e4cbc9dd7991b8
Successfully built PyExecJS


In [6]:
import translators as ts

Using region Taipei City server backend.



In [ ]:
print(ts.translators_pool)

['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'deepl', 'elia', 'google', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']


In [9]:
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

In [10]:
sample_eng = ts.translate_text(sample)
sample_eng

'According to the "2018 Cause of Death Statistics" released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were compiled, and among the causes of death, pneumonia and Alzheimer\'s disease rose in the rankings, and the number of deaths by suicide also increased.'

텍스트 의미 유지하면서 토큰을 바꾸는 방식으로 트랜스레이트를 사용한다.

In [11]:
sample_aug = ts.translate_text(sample_eng, to_language='ko')
sample_aug

"통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 순위에 올랐고 자살로 인한 사망자도 증가했다."

In [12]:
sample1 = ts.translate_text(sample_aug, translator = 'bing', to_language='en')
sample1

'According to the "2018 Cause of Death Statistics" released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were compiled, and among the causes of death, pneumonia and Alzheimer\'s disease ranked first, and deaths due to suicide also increased.'

In [20]:
sample2 = ts.translate_text(sample1, translator = 'bing', to_language='ko')
sample2

"통계청이 발표한 '2018년 사망 원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계가 집계된 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 1위를 차지했으며 자살로 인한 사망도 증가했다."

In [18]:
sample3 = ts.translate_text(sample2, translator = 'modernMt', to_language='en')
sample3

'According to the "2018 Cause of Death Statistics" released by Statistics Korea, the total number of deaths last year was the highest since related statistics were compiled, pneumonia and Alzheimer\'s disease ranked first among the causes of death, and deaths due to suicide also increased.'

In [22]:
sample4 = ts.translate_text(sample2, translator = 'papago', to_language='ko')
print(sample4)
print(sample_aug)
print(sample2)
print(sample)

통계청이 발표한 '2018년 사망 원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계가 집계된 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 1위를 차지했으며 자살로 인한 사망도 증가했다.
통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 순위에 올랐고 자살로 인한 사망자도 증가했다.
통계청이 발표한 '2018년 사망 원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계가 집계된 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 1위를 차지했으며 자살로 인한 사망도 증가했다.
통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. 


https://pypi.org/project/translators/#getting-started

###implementation
1. translator 간의 성능 비교
- 좋은 성능 : modernMt > bing > papago
- 별로 : cloudTranslation, translateCom,

2. 활용 언어 간의 성능 비교


3. 위 샘플 텍스트 뭉치를 5배로 증강해 보기

In [23]:
translator = ts.translators_pool
translator

['alibaba',
 'apertium',
 'argos',
 'baidu',
 'bing',
 'caiyun',
 'cloudTranslation',
 'deepl',
 'elia',
 'google',
 'iciba',
 'iflytek',
 'iflyrec',
 'itranslate',
 'judic',
 'languageWire',
 'lingvanex',
 'niutrans',
 'mglip',
 'mirai',
 'modernMt',
 'myMemory',
 'papago',
 'qqFanyi',
 'qqTranSmart',
 'reverso',
 'sogou',
 'sysTran',
 'tilde',
 'translateCom',
 'translateMe',
 'utibet',
 'volcEngine',
 'yandex',
 'yeekit',
 'youdao']

In [28]:
for i in translator:
    try:
        print(i, ': ' , ts.translate_text(sample1, i, to_language='ko'))

    except:
        pass

argos :  국가 통계청에 의해 발표 된 "2018 사망 통계 원인"에 따르면 작년의 총 사망 수는 관련 통계가 컴파일 된 이래로 가장 높았으며 사망, 폐렴 및 알츠하이머 병의 원인 중 하나가 먼저 순위를 매기고 자살로 인한 사망률이 증가했습니다.
bing :  통계청이 발표한 '2018년 사망 원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계가 집계된 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 1위를 차지했으며 자살로 인한 사망도 증가했다.
caiyun :  According to the "2018 Cause of Death Statistics" released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were compiled, and among the causes of death, pneumonia and Alzheimer's disease ranked first, and deaths due to suicide also increased.
cloudTranslation :  통계청이 발표한 ' 2018년 사망원인통계 '에 따르면 지난해 전체 사망자 수는 관련 통계가 집계된 이후 가장 많았으며 사망원인 중 폐렴과 알츠하이머병이 1위를 차지했고 자살로 인한 사망자도 증가했다.
google :  National Statistics Office에서 발표 한 "2018 사망 통계의 원인"에 따르면, 작년 총 사망자 수는 관련 통계가 수집 된 이후로 가장 높았으며, 사망 원인 중에서 폐렴과 알츠하이머 병이 첫 번째로 순위가 매겨졌습니다. 자살로 인해 증가했습니다.
iciba :  국가통계국이 발표한 '2018년 사망 원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계 작성 이래 가장 많았다. 사망 원인 중 폐렴과 알츠하이머병이 1위를 차지했고 자살 사망자도 증가했다.
itra

/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(nl instead)] with [judic]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')
/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(fr instead)] with [languageWire]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')
/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh-Hans_CN instead)] with [lingvanex]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')
/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh instead)] with [mglip]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')


modernMt :  통계청이 발표한 '2018년 사망원인통계' 에 따르면 지난해 전체 사망자 수는 관련 통계가 집계된 이후 가장 많았고, 사망원인 중 폐렴과 알츠하이머병이 1위를 차지했으며, 자살로 인한 사망자도 증가했다.


/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh-CN instead)] with [myMemory]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')


papago :  통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 전체 사망자 수는 관련 통계 작성 이래 가장 많았고, 사망원인 중 폐렴과 알츠하이머병이 1위를 차지했고, 자살로 인한 사망도 증가했다.


...

reverso :  통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 사망자는 관련 통계가 집계된 이후 가장 많았고 사망원인 중 폐렴 알츠하이머병 등이 1위를 차지했으며 자살로 인한 사망도 증가했다.
sogou :  국가통계청이 발표한' 2018 년 사망 원인 통계' 에 따르면 지난해 사망자 총수는 관련 통계 편찬 이후 최고치를 기록했고, 사망 원인 가운데 폐렴과 알츠하이머병이 1 위를 차지했고 자살로 인한 사망자 수도 증가했다.


/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh instead)] with [sysTran]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')


sysTran :  According to the "2018 Cause of Death Statistics" released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were compiled, and among the causes of death, pneumonia and Alzheimer's disease ranked first, and deaths due to suicide also increased.
translateCom :  통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계가 집계된 이래 가장 많았고,


/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh instead)] with [utibet]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')
/usr/local/lib/python3.10/dist-packages/translators/server.py:185: UserWarning: Unsupported [from_language=auto(zh instead)] with [yeekit]! Please specify it.
  warnings.warn(f'{warn_tips} Please specify it.')


youdao :  据统计厅发布的《2018年死因统计》显示，去年的死亡人数是有相关统计以来的最高水平，在死亡原因中，肺炎和阿尔茨海默病排名第一，自杀死亡人数也有所增加。
